In [12]:
import pandas as pd

In [13]:
restaurant_url = "https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json"

In [14]:
restaurant_url = 'restaurant_data.json'
restaurant_df = pd.read_json(restaurant_url)

In [15]:
excel_file_path = 'Country-Code.xlsx'
country_df = pd.read_excel(excel_file_path)

In [ ]:
restaurants_data = pd.json_normalize(restaurant_df['restaurants'])

total_count = []
missing_restaurants = []
country_matching = []
rating_test = []
other_ratings = []

# iterate through each row in the dataframe
for index, row in restaurants_data.iterrows():
    count = 0
    for restaurant_dict in row:
        if isinstance(restaurant_dict, dict): 
            count += 1
            country_id = restaurant_dict.get('restaurant.location.country_id', None)
            city = restaurant_dict.get('restaurant.location.city', None)
            restaurant_id = restaurant_dict.get('restaurant.R.res_id', None)
            restaurant_name = restaurant_dict.get('restaurant.name', None)
            country_id = restaurant_dict.get('restaurant.location.country_id', None)
            city = restaurant_dict.get('restaurant.location.city', None)
            user_rating_votes = restaurant_dict.get('restaurant.user_rating.votes', None)
            aggregate_rating = restaurant_dict.get('restaurant.user_rating.aggregate_rating', None)
            cuisines = restaurant_dict.get('restaurant.cuisines', None)
            rating_text = restaurant_dict.get('restaurant.user_rating.rating_text')
            
            country_matching.append({
                'Country ID': country_id,
                'City': city,
            })
            
            if rating_text == 'Poor':
                rating_test.append({
                    'Restaurant ID': restaurant_id,
                    'Restaurant Name': restaurant_name,
                    'Country ID': country_id,
                    'City': city,
                    'Rating Score': aggregate_rating,
                    'Rating text': rating_text,
                })

            if rating_text not in ("Excellent", "Very Good", "Good", "Average", "Poor"):
                other_ratings.append({
                    'Restaurant Name': restaurant_name,
                    'Country ID': country_id,
                    'City': city,
                    'Rating text': rating_text
                })

        else:
            missing_restaurants.append(index)
            break
    total_count.append({
    index : count})


# this gives the total breakdown of the number of restaurants
# there are 79 items in the root when inspecting the JSON file, of which in each item there exists
# - results_found (int) , results_start (int) , results_shown (int), restaurants (nested)
total_count

In [ ]:
#these are the missing index with no results_shown
missing_restaurants

In [ ]:
#after doing a left join with the excel containing country code, we can see that there are 20 rows with the Country ID = 17 that cannot be matched
#they also have city = "Dummy"
#therefore in our final csv we excluded these rows by doing an inner join instead
country_match_df = pd.DataFrame(country_matching)
merged_df = pd.merge(country_match_df, country_df, left_on='Country ID', right_on='Country Code', how = 'left')
null_country_df = merged_df[merged_df['Country'].isnull()]
null_country_df

In [ ]:
#found the 1 poor rating to be odd and wanted to isolate it
rating_test

In [ ]:
#shows the other ratings that were not included 
other_ratings_df = pd.DataFrame(other_ratings)
other_ratings_df